In [1]:
# Imports and Initializations

import portion as P
import numpy as np
import matplotlib.pyplot as plt
import cmath
from parfor import parfor
import csv
filestart='./food/nofood10agentswallblind_40samp_20velp_20dirp'
filename=filestart+str(np.random.randint(1000))

In [2]:
# Setting model parameters
v0=2
delv=1
minvel=0
maxvel=4
deltheta=np.pi/8
numagents=10
tau=4
agentradius=1
boundaryradius=40
thresh=12
numsensors=40
num_timesteps=100

numsamp=50

# Initializing agents
agentsposes=np.random.random(numagents)*20+np.random.random(numagents)*20j-10-10j
agentsdirs=np.random.normal(scale=1*deltheta,size=numagents)%(2*np.pi)+np.pi/4
agentsvels=v0*np.ones(numagents)
# agentsposes=np.array([0,20],complex)
# agentsdirs=np.array([0.5,0])
# agentsvels=[v0,0]

# functions related to sensors/boundaries
nprect=np.vectorize(cmath.rect)

def distance_from_circular_boundary(agentpos):
    return boundaryradius-np.abs(agentpos)

def plotboundary(boundaryradius):
    angles=np.linspace(0,2*np.pi,100)
    plt.scatter(np.real(nprect(boundaryradius,angles)),np.imag(nprect(boundaryradius,angles)))

def thetainterval(a,b):
    if abs(b-a)>=2*np.pi:
        return P.closed(0,2*np.pi)
    a=a%(2*np.pi)
    b=b%(2*np.pi)
    if a<=b:
        return P.closed(a,b)
    else:
        return P.closed(a,2*np.pi)|P.closed(0,b)

sensorborders=np.linspace(0,2*np.pi,numsensors+1)
halflensensor=np.pi/(1*numsensors)
sensors=[]
for sensor in range(numsensors):
    sensors.append(thetainterval(sensorborders[sensor],sensorborders[sensor+1]))

In [3]:

def evolve(agentpos,agentdir, agent, othersposesovertime, move, t,tau): # Returns accessible future states estimating states for tau-t future steps
    v=move[0]
    othersposes=othersposesovertime[t]
    newagentdir=(move[1])%(2*np.pi)
    newagentposdir=[updatepos(agentpos,newagentdir,v),newagentdir]
    if not(checkcollisions(newagentposdir[0],agent,othersposes)):
        if t==tau:
            return [getstate(newagentposdir,agent,othersposes)]
        return [getstate(newagentposdir,agent,othersposes)]+sum([evolve(newagentposdir[0],newagentposdir[1],agent,othersposesovertime,nextmove,t+1,tau) for nextmove in make_moveset(newagentposdir[0],newagentposdir[1],v)],[])
    else:
        return []


def checkcollisions(agentpos,agent,othersposes): #checks for collisions with other agents
    if numagents>1 and (((np.abs(np.delete(othersposes,agent)-agentpos)).min()<=2*agentradius) or (distance_from_circular_boundary(agentpos)<=1*agentradius)):
        return True
    elif numagents==1 and distance_from_circular_boundary(agentpos)<=1*agentradius:
        return True
    else:
        return False

    
def updatepos(agentpos,agentdir,v):
    return agentpos+cmath.rect(v,agentdir)

def getstate(agentposdir, agent, othersposes):
    stateout=[]
    agentpos=agentposdir[0]
    agentdir=agentposdir[1]
    # sensorout=wallinterval(agentpos,agentdir,thresh)  #change this to sense wall
    sensorout=P.empty()
    for others in range(numagents):
        if others==agent:
            continue
        else:
            disp=othersposes[others]-agentpos
            radbydisp=agentradius/abs(disp)
            halfsub=np.arcsin(radbydisp) if radbydisp<1 else np.pi/2
            relthet=(cmath.phase(disp)-agentdir)            
            sensorout=sensorout|thetainterval(relthet-halfsub,relthet+halfsub)
    for sensor in sensors:
        if intervallen(sensorout&sensor)>halflensensor:
            stateout.append(1)
        else:
            stateout.append(0)
    return tuple(stateout)

def getbest(stateslists):
    outs=[]
    for statelist in stateslists:
        statematrix=np.array(list(set(statelist)),int)
        if statematrix.shape[0]==0:
            score=-1
        else:
            score=(statematrix.shape[1]-statematrix@statematrix.T-(1-statematrix)@(1-statematrix).T).sum()/(statematrix.shape[0])
        outs.append(score)
    return randargmax(np.array(outs))
    
def randargmax(b):
    return np.random.choice(np.where(b == b.max())[0])

def intervallen(inter):
    length=0
    for i in inter:
        length+=i.upper-i.lower
    return length

def findposesovertime(agentsposes,agentsdirs,agentsvels,tau):
    posesovertime=[]
    posesovertime.append(agentsposes)
    newposes=agentsposes.copy()
    newdirs=agentsdirs.copy()
    for i in range(tau):
        newposes=newposes+nprect(agentsvels,newdirs)
        for agent in range(numagents):
            newposes[agent],newdirs[agent]=reflect_circularwall(newposes[agent],newdirs[agent],0)
        posesovertime.append(newposes.copy())
    return posesovertime

def reflect_circularwall(pos,dir,dep):
    if distance_from_circular_boundary(pos)>1*agentradius:
        return pos,dir
    else:
        xa,ya=np.real(pos),np.imag(pos)
        if not(np.isclose(dir,np.pi/2) or np.isclose(dir,3*np.pi/2)):
            a=1+(np.tan(dir))**2
            b=-2*np.tan(dir)*(xa*np.tan(dir)-ya)
            c=ya**2+(xa*np.tan(dir))**2-2*ya*xa*np.tan(dir)-(boundaryradius-1*agentradius)**2
            x1=(-b-(b**2-4*a*c)**0.5)/(2*a)
            y1=np.tan(dir)*(x1-xa)+ya
            x2=(-b+(b**2-4*a*c)**0.5)/(2*a)
            y2=np.tan(dir)*(x2-xa)+ya
        else:
            x1=xa
            x2=xa
            y1=((boundaryradius-1*agentradius)**2-xa**2)**0.5
            y2=-y1
        distances=np.array([(xa-x1)**2+(ya-y1)**2,(xa-x2)**2+(ya-y2)**2])
        minind=np.argmin(distances)
        xp,yp=[(x1,y1),(x2,y2)][minind]
        distance=(distances[minind])**0.5
        if dep<3:
            newdir=(-1*dir+2*cmath.phase(xp+1j*yp)+np.pi)%(2*np.pi)
        else:
            newdir=(-1*dir+3*cmath.phase(xp+1j*yp)+(1)*np.pi)%(2*np.pi)
        newpos=xp+1j*yp+cmath.rect(distance,newdir)
        return reflect_circularwall(newpos,newdir,dep+1)

def rightdiff(a,b):
    if b>a:
        return a+2*np.pi-b
    else: return a-b

def wallinterval(agentpos,agentdir,thresh):
    dist=distance_from_circular_boundary(agentpos)
    if dist>=thresh:
        return P.empty()
    else:
        x0,y0=(np.real(agentpos),np.imag(agentpos))
        R=boundaryradius
        a=x0**2+y0**2
        b=-1*x0*(a-thresh**2+R**2)
        c=((a-thresh**2+R**2)/2)**2-(y0*R)**2
        x1=(-b+(b**2-4*a*c)**0.5)/(2*a)
        if np.isclose(y0,0):
            if x0>0:
                y1=-1*(R**2-x1**2)**0.5
            else:
                y1=1*(R**2-x1**2)**0.5
        else:
            y1=((a-thresh**2+R**2)/2-x1*x0)/y0

        theta1=(cmath.phase(x1+y1*1j-agentpos))%(2*np.pi)
        theta0=(cmath.phase(agentpos))%(2*np.pi)
        deltheta=rightdiff(theta0,theta1) if (theta0<=np.pi or np.isclose(theta0,2*np.pi)) else -rightdiff(theta1,theta0)
        if (theta0<=np.pi or np.isclose(theta0,2*np.pi)):
            return thetainterval(theta1-agentdir,theta1+2*deltheta-agentdir)
        else:
            return thetainterval(theta1+2*deltheta-agentdir,theta1-agentdir)

def make_moveset(agentpos,agentdir,v):
    dircombs=[(agentdir+deltheta)%(2*np.pi),agentdir,(agentdir-deltheta)%(2*np.pi)]
    tempvelcombs=[v-delv,v,v+delv]
    velcombs=[]
    for vel in tempvelcombs:
        if vel>=minvel and vel<=maxvel:
            velcombs.append(vel)
    moveset=[]
    for vel in velcombs:
        for dir in dircombs:
            moveset.append([vel,dir])
    return moveset

def randevolve(numsamp,agentpos,agentdir, agent, othersposesovertime, move, k,tau,velprobs,thetprobs):
    #k is not used, can be removed
    v=move[0]
    othersposes=othersposesovertime[1]
    newagentdir=(move[1])%(2*np.pi)
    newagentposdir=[updatepos(agentpos,newagentdir,v),newagentdir]
    outposes=[]
    if not(checkcollisions(newagentposdir[0],agent,othersposes)):
        outposes.append(getstate(newagentposdir,agent,othersposes))
        for samp in range(numsamp):
            vel=v
            pos,dir=newagentposdir
            for t in range(2,tau+1):
                newvel=vel+np.random.choice([0,-delv,delv],p=velprobs)
                newdir=(dir+np.random.choice([0,-deltheta,deltheta],p=thetprobs))%(2*np.pi)
                if newvel>maxvel or newvel<minvel:
                    newvel=vel
                newpos=updatepos(pos,newdir,newvel)
                if not(checkcollisions(newpos,agent,othersposesovertime[t])):
                    pos=newpos
                    dir=newdir
                    vel=newvel
                    outposes.append(getstate([pos,dir],agent,othersposesovertime[t]))
                else:
                    break
        return outposes            
    else:
        return []

In [7]:
file=open(filename+'.csv','a')
writer=csv.writer(file)
xlocs=np.real(agentsposes)
ylocs=np.imag(agentsposes)
plt.figure()
plt.gca().set_aspect(1)
plotboundary(boundaryradius)
plt.scatter(np.real(agentsposes),np.imag(agentsposes),c=range(numagents),cmap='gist_ncar',vmin=0,vmax=numagents)
writer.writerow(list(agentsposes))
writer.writerow(list(agentsdirs))
for i in range(numagents):
    dir=cmath.rect(1,agentsdirs[i])
    plt.arrow(xlocs[i],ylocs[i],3*np.real(dir),3*np.imag(dir))
plt.savefig(filename+'_'+str(0)+'.png',facecolor='white')
plt.close()


for time in range(num_timesteps):
    agentsposesovertime=findposesovertime(agentsposes,agentsdirs,agentsvels,tau)
    agentsposes=agentsposesovertime[0]
    print(time)
    chosenmoves=[]
    @parfor(range(numagents))
    def paragent(focal):
        stateslists=[]
        agentpos=agentsposes[focal]
        agentdir=agentsdirs[focal]
        agentvel=agentsvels[focal]
        moveset=make_moveset(agentpos,agentdir,agentvel)
        for move in moveset:
            # evolvelist=evolve(agentpos,agentdir, focal, agentsposesovertime, move, 1,tau)
            evolvelist=randevolve(numsamp,agentpos,agentdir,focal,agentsposesovertime,move,1,tau,[1/3]*3,[1/3]*3)
            stateslists.append(evolvelist)
        chosen=getbest(stateslists)
        return moveset[chosen]
    chosenmoves=paragent
    for updateagent in range(numagents):
        newpos=updatepos(agentsposes[updateagent],chosenmoves[updateagent][1],chosenmoves[updateagent][0])
        if np.abs(newpos)>boundaryradius-agentradius:
            newpos=agentsposes[updateagent]
        agentsposes[updateagent]=newpos
        agentsdirs[updateagent]=chosenmoves[updateagent][1]
        agentsvels[updateagent]=chosenmoves[updateagent][0]
    xlocs=np.real(agentsposes)
    ylocs=np.imag(agentsposes)
    writer.writerow(chosenmoves)
    writer.writerow(list(agentsposes))
    writer.writerow(list(agentsdirs))
    plt.figure()
    plotboundary(boundaryradius)
    plt.gca().set_aspect(1)
    plt.scatter(xlocs,ylocs,c=range(numagents),cmap='gist_ncar',vmin=0,vmax=numagents)
    for i in range(numagents):
        dir=cmath.rect(1,agentsdirs[i])
        plt.arrow(xlocs[i],ylocs[i],3*np.real(dir),3*np.imag(dir))
    plt.savefig(filename+'_'+str(time+201)+'.png',facecolor='white')
    plt.close()
file.close()

0


  0%|          | 0/10 [00:00<?, ?it/s]

1


  0%|          | 0/10 [00:00<?, ?it/s]

2


...

  0%|          | 0/10 [00:00<?, ?it/s]

98


  0%|          | 0/10 [00:00<?, ?it/s]

99


  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
# import matplotlib
# fig, ax = plt.subplots()
# x=np.real(agentsposes)
# y=np.imag(agentsposes)
# plt.xlim(-40,40)
# plt.ylim(-40,40)
# plt.gca().set_aspect(1)
# cmap = matplotlib.colormaps['tab10']
# circles = [plt.Circle((xi,yi), radius=1,facecolor='green') for xi,yi in zip(x,y)]
# # c = matplotlib.collections.PatchCollection(circles)
# for i in range(len(circles)):
#     circles[i].set_facecolor(cmap(i))
#     ax.add_patch(circles[i])
# for s in sensorborders:
#     plt.plot([-40,40],np.tan(s+agentsdirs[0])*(np.array([-40,40])-np.real(agentsposes[0]))+np.imag(agentsposes[0]))
# plt.plot([-40,40],np.tan(s+agentsdirs[0])*(np.array([-40,40])-np.real(agentsposes[0]))+np.imag(agentsposes[0]),'g')

In [5]:
filename

'./rand_evolve/10agentswallsense_50samp_20velp_20dirp825'